# Training NIPs with Improved Manipulation Detection Capabilities

This notebook implements a simple training procedure which jointly optimizes a forensics network and a full neural imaging pipeline to improve manipulation detection capabilities. The current pipeline optimizes forensic analysis in challenging near-real-world conditions, i.e., after downsampling and JPEG compression which are often employed when posting images online. The overall model architecture is shown in detail below:

![manipulation-training](docs/manipulation_detection_training_architecture.png)

For more information, please refer to the following papers:

**References:**

1. P. Korus, N. Memon, *Content Authentication for Neural Imaging Pipelines: End-to-end Optimization of Photo Provenance in Complex Distribution Channels*, CVPR'19, [arxiv:1812.01516](https://arxiv.org/abs/1812.01516) 
2. P. Korus, N. Memon, *Neural Imaging Pipelines - the Scourge or Hope of Forensics?*, 2019, [arXiv:1902.10707](https://arxiv.org/abs/1902.10707)

This notebook was downloaded from: https://github.com/pkorus/neural-imaging

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import gc
import sys
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import scipy as sp
import imageio
import os
import json
import tqdm
import pprint
import matplotlib.pylab as plt

from collections import deque, OrderedDict 
from skimage.measure import compare_ssim, compare_psnr, compare_mse

# Load my TF models
from models import pipelines
from models.forensics import FAN
from models.jpeg import DJPG

# Helper functions
from helpers import coreutils, tf_helpers, validation, loading, plotting

# Training functions
from train_manipulation import construct_models, train_manipulation_nip

# Plotting results
from results import display_results

## Setup the Model

In [ ]:
nip_model = 'UNet'

# Construct the TF model
tf_ops, distribution = construct_models(nip_model, distribution_jpeg=50, loss_metric='L2', jpeg_approx='soft')

print('\n# TF objects')
pprint.pprint(tf_ops)

print('\n# TF distribution channel')
pprint.pprint(distribution)

### Show tensorboard visualization of the model

In [ ]:
from helpers import tf_helpers
tf_helpers.show_graph(tf.get_default_graph().as_graph_def())

## Model Validation (A): Show photo manipulation variants BEFORE content distribution

In [ ]:
# Test the constructed pipeline - show various post-processed versions of a patch

camera_name = 'Nikon D90'
patch_size = 128

# Load the camera model
tf_ops['nip'].load_model(camera_name=camera_name, out_directory_root='./data/raw/nip_model_snapshots')

# Load a sample image and cut a random patch
sample_x = np.load(os.path.join('./data/raw/nip_training_data/', camera_name, 'r47fff40at.npy'))

H, W = sample_x.shape[0:2]

xx = np.random.randint(0, W - patch_size)
yy = np.random.randint(0, H - patch_size)

sample_x = np.expand_dims(sample_x, axis=0)
sample_x = sample_x[:, yy:yy+patch_size, xx:xx+patch_size, :].astype(np.float32) / (2**16 - 1)

# Run the patch through the network
y_patches = tf_ops['sess'].run(tf_ops['operations'], feed_dict={tf_ops['nip'].x: sample_x})

# Plot the images
fig = plotting.imsc(y_patches, ['{}: {{}}'.format(x) for x in distribution['forensics_classes']], ncols=len(distribution['forensics_classes']), figwidth=20)

## Model Validation (B): Show photo manipulation variants AFTER content distribution

In [ ]:
# Test the constructed pipeline - show various post-processed versions of a patch after the distribution channel

# Run the patch through the network
y_patches = tf_ops['sess'].run(tf_ops['fan'].x, feed_dict={tf_ops['nip'].x: sample_x})

# Plot the images
fig = plotting.imsc(y_patches, ['{}+chan. {{}}'.format(x) for x in distribution['forensics_classes']], ncols=len(distribution['forensics_classes']), figwidth=20)

## Training

In [ ]:
# This cell sets up training variables

# Output directory
root_dir = './data/raw/train_manipulation_softjpeg_t/'

# Training setup
training = {
    'camera_name': 'Nikon D90',
    'use_pretrained_nip': True,
    'n_epochs': 5001,
    'learning_rate': 1e-4,
    'run_number': 0,
    'nip_weight': 0.1
}

valid_patch_size = 128
n_patches = 100

# Load the dataset for the given camera
data_directory = os.path.join('./data/raw/nip_training_data/', camera_name)

# Load training / validation data
if 'data' not in globals() or ('camera' not in data or data['camera'] != camera_name):
    # Find available images
    files, val_files = loading.discover_files(data_directory)
        
    # Load training / validation data
    data = {
        'training': loading.load_images(files, data_directory, load='xy'),
        'validation': loading.load_patches(val_files, data_directory, valid_patch_size, n_patches, discard_flat=True, load='xy')
    }
    data['camera'] = camera_name
else:
    print('Using pre-loaded data for {}'.format(data['camera']))

# Set up an example training directory (will be replaced once training is completed)
output_directory = './data/raw/train_manipulation/Nikon D90/UNet/lr-0.1000/000/models/'

In [ ]:
# The actual training
output_directory = train_manipulation_nip(tf_ops, training, distribution, data, {'root': root_dir})

## Compute Confusion Matrix (Online)

### If not trained, load the NIP and FAN models from a checkpoint

In [ ]:
# If not trained, load the NIP and FAN models from a checkpoint
tf_ops['nip'].load_model(camera_name=camera_name, out_directory_root=os.path.join(output_directory, '{nip-model}'))
tf_ops['fan'].load_model(os.path.join(output_directory, 'FAN'))

In [ ]:
from helpers.validation import confusion

# Create a function which generates labels for each batch
def batch_labels(batch_size, n_classes):
    return np.concatenate([x * np.ones((batch_size,), dtype=np.int32) for x in range(n_classes)])

n_classes = len(distribution['forensics_classes'])

conf_mat = confusion(tf_ops['fan'], data['valid_x'][::10,:,:,:], lambda x: batch_labels(x, n_classes))

print(conf_mat.round(2))
print('\nAccuracy: {:.2f}'.format(np.mean(np.diag(conf_mat))))

## Show Training Progress and Confusion Matrix (from Cached Stats)

In [ ]:
class ResultSpec(object):
    
    def __init__(self, plot):
        self.plot = plot
        self.nips = [type(tf_ops['nip']).__name__]
        self.cameras = [training['camera_name']]
        self.dir = root_dir
        self.regularization = ['lr-{:.4f}'.format(training['nip_weight'])]
        self.df = None

In [ ]:
display_results(ResultSpec('progress'))

In [ ]:
display_results(ResultSpec('confusion'))

## Show Image Spectrum

### 1. Load a sample image and crop a random patch

In [ ]:
sample_x = np.load(os.path.join('./data/raw/nip_training_data/', camera_name, 'r47fff40at.npy'))

H, W = sample_x.shape[0:2]

xx = np.random.randint(0, W - patch_size)
yy = np.random.randint(0, H - patch_size)

sample_x = np.expand_dims(sample_x, axis=0)
sample_x = sample_x[:, yy:yy+patch_size, xx:xx+patch_size, :].astype(np.float32) / (2**16 - 1)

### 2. Compare FFT spectra of an RGB patch developed by a pre-trained and an optimized NIP model

In [ ]:
from test_nip_compare import fft_log_norm, nm

# Load the pre-trained NIP model
tf_ops['nip'].load_model(camera_name=camera_name, out_directory_root='./data/raw/nip_model_snapshots')
y_patches = tf_ops['sess'].run(tf_ops['nip'].y, feed_dict={tf_ops['nip'].x: sample_x})

# Load the optimized NIP model
tf_ops['nip'].load_model(camera_name=camera_name, out_directory_root=os.path.join(output_directory, '{nip-model}'))
y_patches_opt = tf_ops['sess'].run(tf_ops['nip'].y, feed_dict={tf_ops['nip'].x: sample_x})

fft_diff = nm(fft_log_norm(np.abs(y_patches_opt.squeeze() - y_patches.squeeze())))

fig = plotting.imsc([y_patches, y_patches_opt, fft_diff], ['(A) RGB patch (pre-trained NIP)', '(B) RGB patch (optimized NIP)', 'FFT(|A-B|)'], ncols=3)

## Run FAN Prediction on a Sample Patch

In [ ]:
# Classify different post-processed variations of a sample patch

# Load the NIP and FAN models
tf_ops['nip'].load_model(camera_name=camera_name, out_directory_root=os.path.join(output_directory, '{nip-model}'))
tf_ops['fan'].load_model(os.path.join(output_directory, 'FAN'))

# Fetch processed & distributed patches - as seen by the FAN
# y_patches = tf_ops['sess'].run(tf_ops['operations'], feed_dict={tf_ops['nip'].x: sample_x})
y_patches = tf_ops['sess'].run(tf_ops['fan'].x, feed_dict={tf_ops['nip'].x: sample_x})

# Run the patches through the FAN
predicted_class, confidence = tf_ops['fan'].process_direct(y_patches, with_confidence=True)

# Prepare labels: real_class -> predicted_class (confidence)
labels = ['{} -> {} ({:.2f})'.format(distribution['forensics_classes'][real_class], distribution['forensics_classes'][pred_class], conf) for real_class, (pred_class, conf) in enumerate(zip(predicted_class, confidence))]

# Plot all images
fig = plotting.imsc(y_patches, labels)

fig.show()